# 1. Table Preparation

### 1.1 Install all necessary libraries and modules

In [175]:
#Install lxml parser
!pip install lxml html5lib beautifulsoup4

In [176]:
#import the necessary libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from bs4 import BeautifulSoup         #this package is used to extract data from html files
from urllib.request import urlopen    #as the name suggests, this is used to open URLs

### 1.2 Scrape the table from the Wikipedia page

In [177]:
# Get the html of the page
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html=urlopen(url)

In [178]:
# Create a Beautiful Soup object from the html
soup = BeautifulSoup(html, 'lxml')

In [179]:
# Get the tables from the Wikipedia page and count how many tables there are in the page
tables=soup.find_all('table')
len(tables)

3

In [180]:
# Because there are three tables, I create a "table" object for the first table only
table=tables[0]

In [181]:
# Extract all rows from the table
rows = table.find_all('tr')
type(rows)

bs4.element.ResultSet

In [182]:
# In order to clean the table, I convert the table from the webpage into a pandas dataframe;
# To this end, I get all the table rows in list form first and then convert the list into a dataframe. 

import re

list_rows = []                                                 #
for row in rows:                                               # Iterate through the table rows
    cells = row.find_all('td')                                 # and assign the cells of the rows to the object "cells"
    str_cells = str(cells)                                     # Convert the BeautifulSoup elements to strings
    cleantext = BeautifulSoup(str_cells, "lxml").get_text()    # Remove the html tags from the text
    list_rows.append(cleantext)                                # Append the rows to a list, which will then be converted to a dataframe

In [183]:
df = pd.DataFrame(list_rows)                                   # Convert the list into a Pandas Dataframe
df.head(5)                                                     # Now the web scraping is complete

,0
0,[]
1,"[M1A\n, Not assigned\n, Not assigned\n]"
2,"[M2A\n, Not assigned\n, Not assigned\n]"
3,"[M3A\n, North York\n, Parkwoods\n]"
4,"[M4A\n, North York\n, Victoria Village\n]"


### 1.3 Modify and clean the table

In [184]:
df1 = df[0].str.split('\n', expand=True)   # Split the first column based on the "\n" character
df1.head(2)

,0,1,2,3
0,[],None,None,None
1,[M1A,", Not assigned",", Not assigned",]


In [185]:
df1.drop([3], axis=1, inplace=True)        # The fourth column does not have useful info; hence, delete it
df1.drop([0], axis=0, inplace=True)        # Same for the first row
df1.rename({0:'PostalCode', 1:'Borough', 2:'Neighborhood'},axis=1, inplace=True)  #Rename the columns as requested
df1.head(2)

,PostalCode,Borough,Neighborhood
1,[M1A,", Not assigned",", Not assigned"
2,[M2A,", Not assigned",", Not assigned"


#### Table cleaning

In [186]:
df1['PostalCode']=df1['PostalCode'].str.strip('[')            #Remove the bracket at the beginiing of the first column
df1['Borough']=df1['Borough'].str.strip(', ')                 #Remove the comma and space at the beginiing of the second column
df1['Neighborhood']=df1['Neighborhood'].str.strip(', ')       #and third column
df1.head(5)

,PostalCode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [187]:
df1 = df1[df1.Borough != "Not assigned"]        #Drop all columns with "not Assigned Borough"
df1.head(10)

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M6A,North York,"Lawrence Manor, Lawrence Heights"
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
9,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
10,M1B,Scarborough,"Malvern, Rouge"
12,M3B,North York,Don Mills
13,M4B,East York,"Parkview Hill, Woodbine Gardens"
14,M5B,Downtown Toronto,"Garden District, Ryerson"


In [188]:
#Reset the row indexes
df1.reset_index(drop=True, inplace=True)
df1.head(3)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [189]:
#Here, I check that there are no Neighborhoods with "Not assigned" values
for neighborhood in df1[['Neighborhood']].iterrows():
    if neighborhood=='Not assigned':
        print('Take action')

In [190]:
#Print final table
df1.head(20)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [191]:
df1.shape

(103, 3)